In [1]:
import torch
from torch import nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
with open('../Data/shakespeare.txt', 'r', encoding='utf8') as f:
    text = f.read()

In [3]:
type(text)

str

In [4]:
print(text[:100])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose mi


In [5]:
len(text)

5445609

In [6]:
all_characters = set(text)

In [7]:
all_characters

{'\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '>',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|',
 '}'}

In [8]:
len(all_characters)

84

In [9]:
# num --> letter
decoder = dict(enumerate(sorted(all_characters)))
decoder

{0: '\n',
 1: ' ',
 2: '!',
 3: '"',
 4: '&',
 5: "'",
 6: '(',
 7: ')',
 8: ',',
 9: '-',
 10: '.',
 11: '0',
 12: '1',
 13: '2',
 14: '3',
 15: '4',
 16: '5',
 17: '6',
 18: '7',
 19: '8',
 20: '9',
 21: ':',
 22: ';',
 23: '<',
 24: '>',
 25: '?',
 26: 'A',
 27: 'B',
 28: 'C',
 29: 'D',
 30: 'E',
 31: 'F',
 32: 'G',
 33: 'H',
 34: 'I',
 35: 'J',
 36: 'K',
 37: 'L',
 38: 'M',
 39: 'N',
 40: 'O',
 41: 'P',
 42: 'Q',
 43: 'R',
 44: 'S',
 45: 'T',
 46: 'U',
 47: 'V',
 48: 'W',
 49: 'X',
 50: 'Y',
 51: 'Z',
 52: '[',
 53: ']',
 54: '_',
 55: '`',
 56: 'a',
 57: 'b',
 58: 'c',
 59: 'd',
 60: 'e',
 61: 'f',
 62: 'g',
 63: 'h',
 64: 'i',
 65: 'j',
 66: 'k',
 67: 'l',
 68: 'm',
 69: 'n',
 70: 'o',
 71: 'p',
 72: 'q',
 73: 'r',
 74: 's',
 75: 't',
 76: 'u',
 77: 'v',
 78: 'w',
 79: 'x',
 80: 'y',
 81: 'z',
 82: '|',
 83: '}'}

In [10]:
# letter --> num
encoder = {char: ind for ind, char in decoder.items()}

In [11]:
encoder

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '&': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '<': 23,
 '>': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'Z': 51,
 '[': 52,
 ']': 53,
 '_': 54,
 '`': 55,
 'a': 56,
 'b': 57,
 'c': 58,
 'd': 59,
 'e': 60,
 'f': 61,
 'g': 62,
 'h': 63,
 'i': 64,
 'j': 65,
 'k': 66,
 'l': 67,
 'm': 68,
 'n': 69,
 'o': 70,
 'p': 71,
 'q': 72,
 'r': 73,
 's': 74,
 't': 75,
 'u': 76,
 'v': 77,
 'w': 78,
 'x': 79,
 'y': 80,
 'z': 81,
 '|': 82,
 '}': 83}

In [12]:
encoded_text = np.array([encoder[char] for char in text])

In [13]:
encoded_text[:50]

array([ 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, 12,  0,  1,  1, 31, 73, 70, 68,  1, 61, 56, 64,
       73, 60, 74, 75,  1, 58, 73, 60, 56, 75, 76, 73, 60, 74,  1, 78])

In [14]:
decoder[27]

'B'

In [15]:
def one_hot_encoder(encoded_text, num_uni_chars):
    
    # encoded_text --> batch of encoded text
    # num_uni_chars --> len(set(text))
    
    one_hot = np.zeros((encoded_text.size, num_uni_chars))
    
    one_hot = one_hot.astype(np.float32)
    
    one_hot[np.arange(one_hot.shape[0]), encoded_text.flatten()]  = 1.0
    
    one_hot = one_hot.reshape((*encoded_text.shape, num_uni_chars))
    
    return one_hot

In [16]:
arr = np.array([1, 2, 0])
arr

array([1, 2, 0])

In [17]:
one_hot_encoder(arr, 3)

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)

In [18]:
example_text = np.arange(10)

In [19]:
example_text

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [20]:
example_text.reshape((5,-1))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

In [21]:
def generate_batches(encoded_text, samp_per_batch=10, seq_len=50):
    
    # X : encoded text of legth seq_len
    # Y : encoded text shifted by one
    
    # how many chars per batch?
    char_per_batch = samp_per_batch * seq_len
    
    # how many batches can we make, given the len of the encoded text?
    num_batches_avail = int(len(encoded_text)/char_per_batch)
    
    # Cut off the end of the encoded text, that won't fit evenly into a batch
    encoded_text = encoded_text[:num_batches_avail * char_per_batch]

    encoded_text = encoded_text.reshape((samp_per_batch, -1))
    
    for n in range(0, encoded_text.shape[1], seq_len):
        
        x = encoded_text[:, n:n+seq_len]
        
        # zeros array to the same shape as x
        y = np.zeros_like(x)
        
        try:
            y[:, :-1] = x[:, 1:]
            y[:, -1] = encoded_text[:, n+seq_len]
        except:
            y[:, :-1] = x[:, 1:]
            y[:,-1] = encoded_text[:, 0]
        
        yield x,y

In [22]:
sample_text = encoded_text[:20]
sample_text

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [23]:
batch_generator = generate_batches(sample_text, samp_per_batch=2, seq_len=5)

In [24]:
x,y = next(batch_generator)

In [25]:
x

array([[0, 1, 1, 1, 1],
       [1, 1, 1, 1, 1]])

In [26]:
y

array([[1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1]])

In [27]:
class CharModel(nn.Module):
    
    def __init__(self, all_chars, num_hidden=256, num_layers=4, drop_prob=0.5, use_gpu=False):
        
        super().__init__()
        
        self.drop_prob = drop_prob
        self.num_layers = num_layers
        self.num_hidden = num_hidden
        self.use_gpu = use_gpu
        
        self.all_chars = all_chars
        self.decoder = dict(enumerate(sorted(all_chars)))
        self.encoder = {char: ind for ind, char in decoder.items()}
        
        self.lstm = nn.LSTM(len(self.all_chars), num_hidden, num_layers, dropout=drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(drop_prob)
        
        self.fc_linear = nn.Linear(num_hidden, len(self.all_chars))
    
    def forward(self, x, hidden):
        
        lstm_output, hidden = self.lstm(x, hidden)
        
        drop_output = self.dropout(lstm_output)
        
        drop_output = drop_output.contiguous().view(-1, self.num_hidden)
        
        final_out = self.fc_linear(drop_output)
        
        return final_out, hidden
    
    def hidden_state(self, batch_size):
        
        if self.use_gpu:
            
            hidden = (torch.zeros(self.num_layers, batch_size, self.num_hidden).cuda(), torch.zeros(self.num_layers, batch_size, self.num_hidden).cuda())
        else:
            hidden = (torch.zeros(self.num_layers, batch_size, self.num_hidden), torch.zeros(self.num_layers, batch_size, self.num_hidden))
        
        return hidden
    

In [28]:
model = CharModel(all_chars=all_characters,
                  num_hidden=512,
                  num_layers=3,
                  drop_prob=0.5,
                  use_gpu=True)

In [29]:
total_param = []

for p in model.parameters():
    total_param.append(int(p.numel()))

sum(total_param)

5470292

In [30]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [31]:
train_percent = 0.9
train_ind = int(len(encoded_text)*train_percent)

train_data = encoded_text[:train_ind]
val_data = encoded_text[train_ind:]

len(train_data), len(val_data)

(4901048, 544561)

In [34]:
# VARIABLES

epochs = 25
batch_size = 128

seq_len = 100

tracker = 0
num_char = max(encoded_text) + 1


In [35]:
import time
start_time = time.time()

model.train()

if model.use_gpu:
    model.cuda()
    
for i in range(epochs):
    
    hidden = model.hidden_state(batch_size)
    
    for x,y in generate_batches(train_data, batch_size, seq_len):
    
        tracker += 1
        
        x = one_hot_encoder(x, num_char)
        
        inputs = torch.from_numpy(x)
        targets = torch.from_numpy(y)
        
        if model.use_gpu:
            inputs = inputs.cuda()
            targets = targets.cuda()
        
        hidden = tuple([state.data for state in hidden])
        
        model.zero_grad()
            
        lstm_output, hidden = model.forward(inputs, hidden)
        loss = criterion(lstm_output, targets.view(batch_size*seq_len).long())
        loss.backward()
        
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
        
        optimizer.step()
        
        if tracker % 25 == 0:
            
            val_hidden = model.hidden_state(batch_size)
            val_losses = []
            model.eval()
            
            for x,y in generate_batches(val_data, batch_size, seq_len):
                
                x = one_hot_encoder(x, num_char)
        
                inputs = torch.from_numpy(x)
                targets = torch.from_numpy(y)

                if model.use_gpu:
                    inputs = inputs.cuda()
                    targets = targets.cuda()
                
                val_hidden = tuple([state.data for state in val_hidden])
                
                lstm_output, val_hidden = model.forward(inputs, val_hidden)
                val_loss = criterion(lstm_output, targets.view(batch_size*seq_len).long())
                val_losses.append(val_loss.item())
            
            model.train()
            
            print(f"EPOCH: {i} step: {tracker} VAL LOSS: {val_loss.item()}")

print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed
    

EPOCH: 0 step: 25 VAL LOSS: 3.2125515937805176
EPOCH: 0 step: 50 VAL LOSS: 3.181339979171753
EPOCH: 0 step: 75 VAL LOSS: 3.0216686725616455
EPOCH: 0 step: 100 VAL LOSS: 2.930293321609497
EPOCH: 0 step: 125 VAL LOSS: 2.7969064712524414
EPOCH: 0 step: 150 VAL LOSS: 2.6831798553466797
EPOCH: 0 step: 175 VAL LOSS: 2.6013193130493164
EPOCH: 0 step: 200 VAL LOSS: 2.519230842590332
EPOCH: 0 step: 225 VAL LOSS: 2.4376440048217773
EPOCH: 0 step: 250 VAL LOSS: 2.360339641571045
EPOCH: 0 step: 275 VAL LOSS: 2.293546199798584
EPOCH: 0 step: 300 VAL LOSS: 2.2196154594421387
EPOCH: 0 step: 325 VAL LOSS: 2.1654555797576904
EPOCH: 0 step: 350 VAL LOSS: 2.1169958114624023
EPOCH: 0 step: 375 VAL LOSS: 2.0793216228485107
EPOCH: 1 step: 400 VAL LOSS: 2.0435314178466797
EPOCH: 1 step: 425 VAL LOSS: 2.0053184032440186
EPOCH: 1 step: 450 VAL LOSS: 1.9778285026550293
EPOCH: 1 step: 475 VAL LOSS: 1.9479742050170898
EPOCH: 1 step: 500 VAL LOSS: 1.9213474988937378
EPOCH: 1 step: 525 VAL LOSS: 1.899275302886963
E

EPOCH: 11 step: 4250 VAL LOSS: 1.3064465522766113
EPOCH: 11 step: 4275 VAL LOSS: 1.3033490180969238
EPOCH: 11 step: 4300 VAL LOSS: 1.2983914613723755
EPOCH: 11 step: 4325 VAL LOSS: 1.3013062477111816
EPOCH: 11 step: 4350 VAL LOSS: 1.3034193515777588
EPOCH: 11 step: 4375 VAL LOSS: 1.3011956214904785
EPOCH: 11 step: 4400 VAL LOSS: 1.301811695098877
EPOCH: 11 step: 4425 VAL LOSS: 1.2997233867645264
EPOCH: 11 step: 4450 VAL LOSS: 1.3011844158172607
EPOCH: 11 step: 4475 VAL LOSS: 1.2964868545532227
EPOCH: 11 step: 4500 VAL LOSS: 1.2991033792495728
EPOCH: 11 step: 4525 VAL LOSS: 1.3001387119293213
EPOCH: 11 step: 4550 VAL LOSS: 1.3006923198699951
EPOCH: 11 step: 4575 VAL LOSS: 1.2931530475616455
EPOCH: 12 step: 4600 VAL LOSS: 1.2955231666564941
EPOCH: 12 step: 4625 VAL LOSS: 1.2982101440429688
EPOCH: 12 step: 4650 VAL LOSS: 1.2955031394958496
EPOCH: 12 step: 4675 VAL LOSS: 1.2970658540725708
EPOCH: 12 step: 4700 VAL LOSS: 1.298282265663147
EPOCH: 12 step: 4725 VAL LOSS: 1.2954405546188354
EP

EPOCH: 21 step: 8375 VAL LOSS: 1.2730283737182617
EPOCH: 21 step: 8400 VAL LOSS: 1.2693971395492554
EPOCH: 22 step: 8425 VAL LOSS: 1.264123797416687
EPOCH: 22 step: 8450 VAL LOSS: 1.2683484554290771
EPOCH: 22 step: 8475 VAL LOSS: 1.2684556245803833
EPOCH: 22 step: 8500 VAL LOSS: 1.2658611536026
EPOCH: 22 step: 8525 VAL LOSS: 1.265004277229309
EPOCH: 22 step: 8550 VAL LOSS: 1.269165277481079
EPOCH: 22 step: 8575 VAL LOSS: 1.2656322717666626
EPOCH: 22 step: 8600 VAL LOSS: 1.2681983709335327
EPOCH: 22 step: 8625 VAL LOSS: 1.2678799629211426
EPOCH: 22 step: 8650 VAL LOSS: 1.2704354524612427
EPOCH: 22 step: 8675 VAL LOSS: 1.272230863571167
EPOCH: 22 step: 8700 VAL LOSS: 1.2699016332626343
EPOCH: 22 step: 8725 VAL LOSS: 1.2670989036560059
EPOCH: 22 step: 8750 VAL LOSS: 1.2690883874893188
EPOCH: 22 step: 8775 VAL LOSS: 1.2643638849258423
EPOCH: 23 step: 8800 VAL LOSS: 1.2673999071121216
EPOCH: 23 step: 8825 VAL LOSS: 1.2664779424667358
EPOCH: 23 step: 8850 VAL LOSS: 1.2656456232070923
EPOCH: 

In [36]:
model_name = "hidden512_layers3_shakes_saul_4.net"

In [37]:
torch.save(model.state_dict(), model_name)